# Interior AI

### Imports

In [2]:
import os
from PIL import Image
import cv2
import numpy as np
import random
import shutil
%matplotlib inline
import matplotlib
import time
import functools
import tensorflow as tf
import matplotlib.pyplot as plt
# Load compressed models from tensorflow_hub
os.environ['TFHUB_MODEL_LOAD_FORMAT'] = 'COMPRESSED'
import tensorflow_hub as hub


from Semantic_Segmentation.segmentation_helper import convert_32bit_to_24bit, convert_masks_to_semantic_labels, rename_files_in_folder,train_test_split
from style_transfer.style_transfer_helper import rgb_semantic_segmentation_mask_to_binary_mask, resize_style_image,nst_load_img,apply_class_specific_style_transfer

## Semantic Segmentation

### Setting up Paths
Change paths to the image directory to the input and output folder

### convert_32bit_to_24bit

In [ ]:
# Uncomment this and add your own path 
# image_dir = "path/to/rgba/images/folder"
# output_dir = "path/to/save/rgb/images/folder"

In [ ]:
convert_32bit_to_24bit(image_dir,output_dir)

In [4]:
# Uncomment this and add your own path
# input_folder = "path/to/ground/truth/segmentation/masks"
# output_folder = "path/to/save/semantic/labels"

### convert_masks_to_semantic_labels

In [ ]:
convert_masks_to_semantic_labels(input_folder,output_folder)

### rename_files_in_folder

In [ ]:
# Uncomment this and add your own path
# image_folder_path = "path/to/folder/containing/images"
# semantic_label_folder_path = "path/to/folder/containing/semantic/label/images"
# new_name_prefix = "image_"
# start_index = 1

In [ ]:
rename_files_in_folder(image_folder_path, new_name_prefix, start_index)
rename_files_in_folder(semantic_label_folder_path, new_name_prefix, start_index)

### train_test_split

In [ ]:
# Uncomment this and add your own path 
# images_folder = '/path/to/images_folder'
# train_images_folder = '/path/to/train_images_folder'
# test_images_folder = '/path/to/test_images_folder'

# semantic_label_images_folder = '/path/to/semantic_label_images'
# train_semantic_label_images_folder = '/path/to/train_images_folder'
# test_semantic_label_images_folder = '/path/to/test_images_folder'

# split_ratio = 0.8
# random_seed = 42

In [ ]:
train_test_split(images_folder, train_images_folder, test_images_folder, split_ratio, random_seed)
train_test_split(semantic_label_images_folder, train_semantic_label_images_folder, test_semantic_label_images_folder, split_ratio, random_seed)

### Install the package

In [ ]:
#! pip install git+https://github.com/divamgupta/image-segmentation-keras

### Initialize the model

In [ ]:
from keras_segmentation.models.unet import vgg_unet

model = vgg_unet(n_classes=11, input_height=320, input_width=640)

### Train the Model

In [ ]:
model.train(
    train_images =  "/path/to/train/images/folder",
    train_annotations = "/path/to/train/masks(semantic_labels)/folder",
    checkpoints_path = "/tmp/vgg_unet_1" , 
    epochs= 70 # No. of epochs as per required
)

### Evaluate the Model

In [ ]:
print(model.evaluate_segmentation( inp_images_dir="/path/to/test/images/folder", annotations_dir="/path/to/test/masks(semantic_labels)/folder" ) )

### RGB values of the classes used in the segmentation predictions

In [ ]:
from keras_segmentation.data_utils.data_loader import class_colors
RGB_Values = [color[::-1] for color in class_colors[:11]]
class_names = ['Wall','Table','Storage','TV Unit','Chair','Sofa','Curtain','Ceiling','Rug','Floor','Others']
for i, color in enumerate(RGB_Values):
    class_name = class_names[i]
    print(f"Class Name: {class_name},RGB Value: {color}")

### Perform Multiple Semantic Segmentation Predictions on a Directory of Input Images and save them in the Output Folder

In [ ]:
output = model.predict_multiple(
    inp_dir = "path/to/test/images/folder",
    out_dir = "path/to/output/folder"
)

### Perform Multiple Overlaying the Segmentation Ouput Predictions on a Directory of Input Images and save them in the Output Folder

In [ ]:
output = model.predict_multiple(
    inp_dir = "path/to/test/images/folder",
    out_dir = "path/to/output/folder",
    overlay_img=True, # set this true if you want to overlay the image
    show_legends=True, # set this true if you want to show class names and their corresponding color along the output image
    class_names = ['Wall','Table','Storage','TV Unit','Chair','Sofa','Curtain','Ceiling','Rug','Floor','Others'] #Class Names
)

## Style Transfer

### rgb_semantic_segmentation_mask_to_binary_mask

In [ ]:
# Uncomment this and add your own path 
# segmented_mask = 'path/to/segmented_mask'
# Class_name = 'Wall'
# save_path = 'path/to/binary_mask.png'

In [ ]:
rgb_semantic_segmentation_mask_to_binary_mask(segmented_mask, Class_name, save_path)

### resize_style_image

In [ ]:
# Uncomment this and add your own path 
# new_width = 1920
# new_height = 1080
# image_path = "/path/to/style/image"

In [ ]:
resize_style_image(image_path,new_width,new_height)

### Neural Style Transfer

In [ ]:
# Uncomment this and add your own path 
# content_image = nst_load_img('path/to/content/image')
# style_image = nst_load_img('path/to/style/image')
# output_image_path = 'path/to/save/neural/style/image'

In [ ]:
# Load the arbitrary image stylization model from TensorFlow Hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Apply image stylization by passing the content and style images to the model
stylized_image = hub_model(tf.constant(content_image), tf.constant(style_image))[0]

# Convert the stylized image tensor to tf.uint8 data type for saving
stylized_image = tf.image.convert_image_dtype(stylized_image, tf.uint8)

# Remove the singleton dimensions and obtain a 3-dimensional image tensor
stylized_image = tf.squeeze(stylized_image)

# Convert the stylized image tensor to a NumPy array
stylized_image = stylized_image.numpy()

# Save the stylized image as a PNG file
plt.imsave(output_image_path, stylized_image)

### apply_class_specific_style_transfer

In [ ]:
# Uncomment this and add your own path 
# input_image = "path/to/input_image"
# binary_mask = "path/to/binary_mask"
# style_image = "path/to/neural_style_image"
# output_path = "path/to/class_style_transfer_image"

In [ ]:
apply_class_specific_style_transfer(input_image, binary_mask, style_image, output_path)